# Metodologia para Avaliação de Overhead em HPC usando DGEMM no Docker

Este _notebook_ organiza a metodologia para quantificar o _overhead_ (sobrecarga), introduzido pelo uso de containers Docker, na execução de aplicações de alto desempenho. Com esta finalidade é utilizada uma aplicação baseadas na operação DGEMM, comparando o desempenho de sua exeução no container com a execução nativa no sistema operacional.

## 1. Introdução

O objetivo deste estudo é quantificar o _overhead_ introduzido pelo Docker em aplicações de alto desempenho, usando como base um benchmark de DGEMM. São consideradas diferentes combinações de:

- Implementações de BLAS (por exemplo, OpenBLAS, BLIS)
- Implementação sequencial e com multiplas _threads_
- Tamanhos de matrizes
- Execução em ambiente **nativo** vs. **Docker**

As métricas coletadas em cada execução são:

- Tempo de execução
- Desempenho em FLOPS ou GFLOPS (Operações de ponto flutuante por segundo)

A partir dessas métricas, serão definidas expressões para calcular o _overhead_ em termos de tempo e desempenho, bem como procedimentos estatísticos para garantir robustez na análise.


In [ ]:
# TODO: carregar e inspecionar dados brutos (por exemplo, a partir de arquivos CSV)
# import pandas as pd
# df = pd.read_csv('resultados_dgemm.csv')
# df.head()

## 2. Desenho Experimental

Os experimentos são organizados conforme os seguintes fatores:

### 2.1 Ambientes comparados
- Execução **nativa** no sistema operacional (host).
- Execução **em container Docker**, mantendo o máximo de equivalência possível:
  - Mesma implementação de BLAS
  - Mesmo compilador e flags de compilação
  - Mesma versão do benchmark

### 2.2 Fatores de variação
- Implementações BLAS: OpenBLAS, BLIS, etc.
- Tamanhos de matriz: 32, 64, ..., 4096, aumentando o tamanho de 32 em 32

### 2.3 Repetições
Para cada combinação de parâmetros (BLAS, número de *threads*, tamanho da matriz, ambiente), o benchmark deve ser executado
múltiplas vezes (por exemplo, \( n = 5 \)), registrando-se o tempo de execução e o desempenho em GFLOPS.

Essas repetições permitem estimar média, desvio padrão, intervalos de confiança e significância estatística das diferenças
entre o ambiente nativo e o Docker.


In [ ]:
# TODO: implementar leitura e organização dos resultados experimentais
# Exemplo: agrupar dados por (BLAS, threads, N, ambiente)
# group_cols = ['blas', 'N', 'ambiente']
# df_grouped = df.groupby(group_cols)
# df_grouped.describe()

## 3. Definição das Métricas de _Overhead_

A avaliação do _overhead_ será feita a partir de duas métricas principais:

- _Overhead_ no tempo de execução
- _Overhead_ no desempenho (GFLOPS)

Para cada configuração $(\text{BLAS}, N)$, serão comparados os resultados nos ambientes **host** (nativo) e **Docker**.

### 3.1 _Overhead_ em tempo de execução

Sejam:

- $( T_{\text{host}} )$: tempo médio de execução no sistema nativo
- $( T_{\text{dock}} )$: tempo médio de execução no Docker

Define-se o _overhead_ percentual em tempo como:

$\text{Overhead}_{\text{tempo}} = \left( \frac{T_{\text{dock}} - T_{\text{host}}}{T_{\text{host}}} \right) \times 100\%.$ 

Também é útil definir o fator de *slowdown*:

$ \text{Slowdown} = \frac{T_{\text{dock}}}{T_{\text{host}}}.$

Valores de *slowdown* maiores que 1 indicam que a execução em Docker é mais lenta que a nativa.

### 3.2 _Overhead_ em desempenho (GFLOPS)

Sejam:

- $ P_{\text{host}} $: desempenho médio em GFLOPS no sistema nativo
- $ P_{\text{dock}} $: desempenho médio em GFLOPS no Docker

Define-se o _overhead_ em termos de desempenho como:

$ \text{Overhead}_{\text{GFLOPS}} = \left( \frac{P_{\text{host}} - P_{\text{dock}}}{P_{\text{host}}} \right) \times 100\%.$

Valores positivos indicam perda de desempenho ao usar Docker; valores próximos de zero indicam diferença desprezível.


In [ ]:
# TODO: calcular overhead de tempo e GFLOPS a partir dos dados agregados
# Exemplo (supondo que já existam colunas T_host, T_dock, P_host, P_dock):
# df_overhead = df_config.copy()
# df_overhead['overhead_tempo_percent'] = (df_overhead['T_dock'] - df_overhead['T_host']) / df_overhead['T_host'] * 100
# df_overhead['slowdown'] = df_overhead['T_dock'] / df_overhead['T_host']
# df_overhead['overhead_gflops_percent'] = (df_overhead['P_host'] - df_overhead['P_dock']) / df_overhead['P_host'] * 100
# df_overhead.head()

## 4. Tratamento Estatístico

Para cada combinação de parâmetros e ambiente, será feita uma análise estatística básica dos resultados,
incluindo cálculo de médias, desvios padrão, intervalos de confiança e testes de significância.

### 4.1 Cálculo de média e variabilidade

Sejam $ x_1, x_2, \ldots, x_n $ as observações (por exemplo, tempos de execução de $ n $ repetições).

A média amostral é dada por:

$ \overline{x} = \frac{1}{n} \sum_{i=1}^{n} x_i,$

e o desvio padrão amostral é:

$ \sigma = \sqrt{ \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \overline{x})^2 }. $

Essas quantidades podem ser calculadas tanto para tempo de execução quanto para GFLOPS, em cada ambiente.

### 4.2 Intervalos de confiança (opcional)

Supondo distribuição aproximadamente normal das médias, um intervalo de confiança de 95% para a média pode ser escrito como:

$ IC = \overline{x} \pm t_{\alpha/2} \cdot \frac{\sigma}{\sqrt{n}}, $

onde $ t_{\alpha/2} $ é o valor crítico da distribuição t de Student.

### 4.3 Testes de significância

Para verificar se as diferenças entre host e Docker são estatisticamente significativas, podem ser utilizados:

- Teste t pareado (se as suposições de normalidade forem razoáveis)
- Testes não-paramétricos, como Wilcoxon, se necessário

Essa análise ajuda a distinguir diferenças reais de flutuações devido a ruído ou variabilidade do sistema.


In [ ]:
# TODO: implementar cálculos estatísticos (médias, desvios, IC, testes de hipótese)
# Exemplo: usar scipy.stats para testes t ou Wilcoxon
# from scipy import stats
# stats.ttest_rel(tempos_host, tempos_docker)

## 5. Apresentação dos Resultados

### 5.1 Tabelas sugeridas

Uma forma clara de apresentar os resultados é usar tabelas que agreguem as métricas principais para cada combinação de parâmetros.
Por exemplo:

| BLAS | Threads | N | $T_{\text{host}}$ | $T_{\text{dock}}$ | Overhead tempo (%) | $P_{\text{host}}$ | $P_{\text{dock}}$ | Overhead GFLOPS (%) |
|------|---------|---|--------------------|--------------------|--------------------|--------------------|--------------------|----------------------|
| ...  |   ...   |...|        ...         |        ...         |        ...         |        ...         |        ...         |         ...          |

### 5.2 Gráficos sugeridos

Alguns gráficos que ajudam a visualizar o comportamento do overhead:

- Overhead de tempo (%) em função do tamanho da matriz \( N \)
- Overhead de tempo (%) em função do número de *threads*
- Comparação do overhead entre diferentes implementações de BLAS

Esses gráficos podem ser construídos tanto para tempo quanto para GFLOPS.


In [ ]:
# TODO: gerar tabelas e gráficos para visualização dos resultados
# Exemplo básico com matplotlib:
# import matplotlib.pyplot as plt
# plt.plot(df_overhead['N'], df_overhead['overhead_tempo_percent'], marker='o')
# plt.xlabel('Tamanho da matriz N')
# plt.ylabel('Overhead de tempo (%)')
# plt.title('Overhead de tempo vs N')
# plt.grid(True)
# plt.show()

## 6. Conclusões Possíveis

Com base nessa metodologia, é possível responder a questões como:

- Existe overhead mensurável ao usar Docker para aplicações de alto desempenho baseadas em DGEMM?
- Esse overhead depende do tamanho da matriz \( N \)?
- O overhead aumenta ou diminui com o número de *threads*?
- Diferentes implementações de BLAS são mais ou menos sensíveis ao uso de Docker?

A combinação de métricas quantitativas (overhead de tempo e de GFLOPS) com análise estatística confere rigor ao estudo
e permite conclusões bem fundamentadas em um contexto de HPC.


In [ ]:
# TODO: sumarizar resultados e gerar artefatos finais para o TCC (tabelas finais, figuras, etc.)
# Exemplo: salvar figuras e tabelas em arquivos para inclusão no texto do TCC
# df_overhead.to_csv('tabelas_overhead.csv', index=False)